In [13]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from alphamind.api import *
from alphamind.strategy.strategy import RunningSetting
from alphamind.strategy.strategy import Strategy
from PyFin.api import *
from PyFin.Math.Accumulators.StatefulAccumulators import MovingAverage
from PyFin.Math.Accumulators.StatefulAccumulators import MovingSharp
from PyFin.Math.Accumulators.StatefulAccumulators import MovingMaxDrawdown

plt.style.use('ggplot')

In [24]:
"""
Back test parameter settings
"""

benchmark_code = 905
universe = Universe("zz800_cyb", ['zz800', 'cyb'])

def create_scenario(weights_bandwidth=0.02, target_vol=0.01, method='risk_neutral'):
    start_date = '2010-01-01'
    end_date = '2018-04-27'
    freq = '10b'
    neutralized_risk = None

    alpha_factors = {
        'f01': CSQuantiles(LAST('ep_q'), groups='sw1_adj'),
        'f02': CSQuantiles(LAST('roe_q'), groups='sw1_adj'),
        'f03': CSQuantiles(LAST('SGRO'), groups='sw1_adj'),
        'f04': CSQuantiles(LAST('GREV'), groups='sw1_adj'),
        'f05': CSQuantiles(LAST('con_peg_rolling'), groups='sw1_adj'),
        'f06': CSQuantiles(LAST('con_pe_rolling_order'), groups='sw1_adj'),
        'f07': CSQuantiles(LAST('IVR'), groups='sw1_adj'),
        'f08': CSQuantiles(LAST('ILLIQUIDITY'), groups='sw1_adj'),
        'f09': CSQuantiles(LAST('DividendPaidRatio'), groups='sw1_adj'),
    }

    weights = dict(
                   f01=0.5,
                   f02=1.,
                   f03=1.,
                   f04=1.,
                   f05=-1.,
                   f06=-0.5,
                   f07=0.5,
                   f08=0.5,
                   f09=0.5
                   )

    alpha_model = ConstLinearModel(features=alpha_factors, weights=weights)

    data_meta = DataMeta(freq=freq,
                         universe=universe,
                         batch=1,
                         neutralized_risk=None,
                         pre_process=None,
                         post_process=None)

    industries = industry_list('sw_adj', 1)
    total_risk_names = ['benchmark', 'total'] + \
                       ['EARNYILD', 'LIQUIDTY', 'GROWTH', 'SIZE', 'BETA', 'MOMENTUM'] + \
                       industries

    b_type = []
    l_val = []
    u_val = []

    for name in total_risk_names:
        if name == 'benchmark':
            b_type.append(BoundaryType.RELATIVE)
            l_val.append(0.8)
            u_val.append(1.0)
        elif name == 'total':
            b_type.append(BoundaryType.ABSOLUTE)
            l_val.append(.0)
            u_val.append(.0)
        elif name == 'EARNYILD':
            b_type.append(BoundaryType.ABSOLUTE)
            l_val.append(0.00)
            u_val.append(0.20)
        elif name == 'GROWTH':
            b_type.append(BoundaryType.ABSOLUTE)
            l_val.append(0.00)
            u_val.append(0.20)
        elif name == 'MOMENTUM':
            b_type.append(BoundaryType.ABSOLUTE)
            l_val.append(0.20)
            u_val.append(0.20)
        elif name == 'SIZE':
            b_type.append(BoundaryType.ABSOLUTE)
            l_val.append(-0.2)
            u_val.append(0.0)
        elif name == 'LIQUIDTY':
            b_type.append(BoundaryType.ABSOLUTE)
            l_val.append(-0.40)
            u_val.append(-0.20)
        elif name not in ["计算机", "医药生物", "国防军工", "信息服务", "机械设备"] and name in industries:
            b_type.append(BoundaryType.RELATIVE)
            l_val.append(0.8)
            u_val.append(1.0)
        elif name in ["计算机", "医药生物", "国防军工", "信息服务", "机械设备"]:
            b_type.append(BoundaryType.ABSOLUTE)
            l_val.append(0.0)
            u_val.append(0.05)
        else:
            b_type.append(BoundaryType.ABSOLUTE)
            l_val.append(0)
            u_val.append(0)

    bounds = create_box_bounds(total_risk_names, b_type, l_val, u_val)
    running_setting = RunningSetting(universe,
                                     start_date,
                                     end_date,
                                     freq,
                                     benchmark=benchmark_code,
                                     weights_bandwidth=weights_bandwidth,
                                     rebalance_method=method,
                                     bounds=bounds,
                                     target_vol=target_vol,
                                     turn_over_target=0.4)

    strategy = Strategy(alpha_model, data_meta, running_setting)
    ret_df, positions = strategy.run()
    return ret_df

In [25]:
def create_report(ret_df, windows):
    sharp_calc = MovingSharp(windows, x='ret', y='riskFree')
    drawdown_calc = MovingMaxDrawdown(windows, x='ret')
    max_drawdown_calc = MovingMaxDrawdown(len(ret_df), x='ret')
    
    ret_df['ret_after_tc'] = ret_df.excess_return - 0.002 * ret_df.turn_over
    res_df = pd.DataFrame(columns=['daily_return', 'cum_ret', 'sharp', 'drawdown', 'max_drawn', 'leverage'])
    total_returns = 0.

    for i, ret in enumerate(ret_df['ret_after_tc']):
        date = ret_df.index[i]
        total_returns += ret
        sharp_calc.push({'ret': ret, 'riskFree': 0.})
        drawdown_calc.push({'ret': ret})
        max_drawdown_calc.push({'ret': ret})

        res_df.loc[date, 'daily_return'] = ret
        res_df.loc[date, 'cum_ret'] = total_returns
        res_df.loc[date, 'drawdown'] = drawdown_calc.result()
        res_df.loc[date, 'max_drawn'] = max_drawdown_calc.result()
        res_df.loc[date, 'leverage'] = ret_df.loc[date, 'leverage']

        if i < 5:
            res_df.loc[date, 'sharp'] = 0.
        else:
            res_df.loc[date, 'sharp'] = sharp_calc.result() * np.sqrt(windows)
    return res_df

In [26]:
weight_gaps = [0.005, 0.010, 0.015, 0.020]

with pd.ExcelWriter(f'{universe.name}_{benchmark_code}_gap.xlsx', engine='xlsxwriter') as writer:
    for i, weight_gap in enumerate(weight_gaps):
        ret_df = create_scenario(weight_gap, target_vol=0.01, method='risk_neutral')
        res_df = create_report(ret_df, 25)
        res_df.to_excel(writer, sheet_name=f'{i}')
        alpha_logger.info(f"weight_gap: {weight_gap} finished")

2018-05-04 22:01:52,871 - ALPHA_MIND - INFO - starting backting ...
2018-05-04 22:02:12,307 - ALPHA_MIND - INFO - alpha factor data loading finished ...
2018-05-04 22:02:13,368 - ALPHA_MIND - INFO - industry data loading finished ...
2018-05-04 22:02:13,757 - ALPHA_MIND - INFO - benchmark data loading finished ...
2018-05-04 22:02:21,004 - ALPHA_MIND - INFO - risk_model data loading finished ...
2018-05-04 22:02:49,000 - ALPHA_MIND - INFO - returns data loading finished ...
d:\ProgramData\Anaconda3\lib\site-packages\alpha_mind-0.1.1-py3.6-win-amd64.egg\alphamind\strategy\strategy.py:106: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  is_in_benchmark = (total_data.weight > 0.).astype(float).reshape((-1, 1))
2018-05-04 22:02:54,655 - ALPHA_MIND - INFO - 2010-01-04 00:00:00 re-balance: 798 codes
2018-05-04 22:02:54,708 - ALPHA_MIND - INFO - 2010-01-18 00:00:00 re-balance: 798 codes
d:\ProgramData\Anaconda3\lib\site-pa

2018-05-04 22:03:07,651 - ALPHA_MIND - INFO - 2013-04-24 00:00:00 re-balance: 899 codes
2018-05-04 22:03:07,826 - ALPHA_MIND - INFO - 2013-05-13 00:00:00 re-balance: 900 codes
2018-05-04 22:03:08,003 - ALPHA_MIND - INFO - 2013-05-27 00:00:00 re-balance: 900 codes
2018-05-04 22:03:08,168 - ALPHA_MIND - INFO - 2013-06-13 00:00:00 re-balance: 900 codes
2018-05-04 22:03:08,333 - ALPHA_MIND - INFO - 2013-06-27 00:00:00 re-balance: 900 codes
2018-05-04 22:03:08,500 - ALPHA_MIND - INFO - 2013-07-11 00:00:00 re-balance: 900 codes
2018-05-04 22:03:08,676 - ALPHA_MIND - INFO - 2013-07-25 00:00:00 re-balance: 900 codes
2018-05-04 22:03:08,843 - ALPHA_MIND - INFO - 2013-08-08 00:00:00 re-balance: 900 codes
2018-05-04 22:03:09,022 - ALPHA_MIND - INFO - 2013-08-22 00:00:00 re-balance: 900 codes
2018-05-04 22:03:09,186 - ALPHA_MIND - INFO - 2013-09-05 00:00:00 re-balance: 900 codes
2018-05-04 22:03:09,353 - ALPHA_MIND - INFO - 2013-09-23 00:00:00 re-balance: 899 codes
2018-05-04 22:03:09,526 - ALPHA_

2018-05-04 22:03:23,770 - ALPHA_MIND - INFO - 2017-03-07 00:00:00 re-balance: 853 codes
2018-05-04 22:03:23,941 - ALPHA_MIND - INFO - 2017-03-21 00:00:00 re-balance: 853 codes
2018-05-04 22:03:24,104 - ALPHA_MIND - INFO - 2017-04-06 00:00:00 re-balance: 853 codes
2018-05-04 22:03:24,273 - ALPHA_MIND - INFO - 2017-04-20 00:00:00 re-balance: 853 codes
2018-05-04 22:03:24,442 - ALPHA_MIND - INFO - 2017-05-05 00:00:00 re-balance: 854 codes
2018-05-04 22:03:24,625 - ALPHA_MIND - INFO - 2017-05-19 00:00:00 re-balance: 854 codes
2018-05-04 22:03:24,795 - ALPHA_MIND - INFO - 2017-06-06 00:00:00 re-balance: 854 codes
2018-05-04 22:03:24,960 - ALPHA_MIND - INFO - 2017-06-20 00:00:00 re-balance: 854 codes
2018-05-04 22:03:25,127 - ALPHA_MIND - INFO - 2017-07-04 00:00:00 re-balance: 855 codes
2018-05-04 22:03:25,299 - ALPHA_MIND - INFO - 2017-07-18 00:00:00 re-balance: 855 codes
2018-05-04 22:03:25,466 - ALPHA_MIND - INFO - 2017-08-01 00:00:00 re-balance: 853 codes
2018-05-04 22:03:25,635 - ALPHA_

2018-05-04 22:04:38,683 - ALPHA_MIND - INFO - 2012-01-31 00:00:00 re-balance: 898 codes
2018-05-04 22:04:38,859 - ALPHA_MIND - INFO - 2012-02-14 00:00:00 re-balance: 898 codes
2018-05-04 22:04:39,023 - ALPHA_MIND - INFO - 2012-02-28 00:00:00 re-balance: 896 codes
2018-05-04 22:04:39,194 - ALPHA_MIND - INFO - 2012-03-13 00:00:00 re-balance: 897 codes
2018-05-04 22:04:39,361 - ALPHA_MIND - INFO - 2012-03-27 00:00:00 re-balance: 896 codes
2018-05-04 22:04:39,534 - ALPHA_MIND - INFO - 2012-04-13 00:00:00 re-balance: 898 codes
2018-05-04 22:04:39,707 - ALPHA_MIND - INFO - 2012-04-27 00:00:00 re-balance: 898 codes
2018-05-04 22:04:39,887 - ALPHA_MIND - INFO - 2012-05-15 00:00:00 re-balance: 898 codes
2018-05-04 22:04:40,054 - ALPHA_MIND - INFO - 2012-05-29 00:00:00 re-balance: 898 codes
2018-05-04 22:04:40,220 - ALPHA_MIND - INFO - 2012-06-12 00:00:00 re-balance: 898 codes
2018-05-04 22:04:40,386 - ALPHA_MIND - INFO - 2012-06-27 00:00:00 re-balance: 898 codes
2018-05-04 22:04:40,555 - ALPHA_

2018-05-04 22:04:54,898 - ALPHA_MIND - INFO - 2015-12-10 00:00:00 re-balance: 869 codes
2018-05-04 22:04:55,070 - ALPHA_MIND - INFO - 2015-12-24 00:00:00 re-balance: 869 codes
2018-05-04 22:04:55,233 - ALPHA_MIND - INFO - 2016-01-08 00:00:00 re-balance: 863 codes
2018-05-04 22:04:55,409 - ALPHA_MIND - INFO - 2016-01-22 00:00:00 re-balance: 863 codes
2018-05-04 22:04:55,586 - ALPHA_MIND - INFO - 2016-02-05 00:00:00 re-balance: 863 codes
2018-05-04 22:04:55,761 - ALPHA_MIND - INFO - 2016-02-26 00:00:00 re-balance: 863 codes
2018-05-04 22:04:55,928 - ALPHA_MIND - INFO - 2016-03-11 00:00:00 re-balance: 863 codes
2018-05-04 22:04:56,092 - ALPHA_MIND - INFO - 2016-03-25 00:00:00 re-balance: 863 codes
2018-05-04 22:04:56,256 - ALPHA_MIND - INFO - 2016-04-11 00:00:00 re-balance: 863 codes
2018-05-04 22:04:56,423 - ALPHA_MIND - INFO - 2016-04-25 00:00:00 re-balance: 863 codes
2018-05-04 22:04:56,605 - ALPHA_MIND - INFO - 2016-05-10 00:00:00 re-balance: 863 codes
2018-05-04 22:04:56,788 - ALPHA_

2018-05-04 22:06:09,652 - ALPHA_MIND - INFO - 2010-11-04 00:00:00 re-balance: 899 codes
2018-05-04 22:06:09,818 - ALPHA_MIND - INFO - 2010-11-18 00:00:00 re-balance: 899 codes
2018-05-04 22:06:09,980 - ALPHA_MIND - INFO - 2010-12-02 00:00:00 re-balance: 899 codes
2018-05-04 22:06:10,141 - ALPHA_MIND - INFO - 2010-12-16 00:00:00 re-balance: 899 codes
2018-05-04 22:06:10,306 - ALPHA_MIND - INFO - 2010-12-30 00:00:00 re-balance: 899 codes
2018-05-04 22:06:10,470 - ALPHA_MIND - INFO - 2011-01-14 00:00:00 re-balance: 899 codes
2018-05-04 22:06:10,632 - ALPHA_MIND - INFO - 2011-01-28 00:00:00 re-balance: 899 codes
2018-05-04 22:06:10,796 - ALPHA_MIND - INFO - 2011-02-18 00:00:00 re-balance: 897 codes
2018-05-04 22:06:10,974 - ALPHA_MIND - INFO - 2011-03-04 00:00:00 re-balance: 898 codes
2018-05-04 22:06:11,139 - ALPHA_MIND - INFO - 2011-03-18 00:00:00 re-balance: 898 codes
2018-05-04 22:06:11,302 - ALPHA_MIND - INFO - 2011-04-01 00:00:00 re-balance: 898 codes
2018-05-04 22:06:11,473 - ALPHA_

2018-05-04 22:06:25,564 - ALPHA_MIND - INFO - 2014-09-16 00:00:00 re-balance: 900 codes
2018-05-04 22:06:25,732 - ALPHA_MIND - INFO - 2014-09-30 00:00:00 re-balance: 900 codes
2018-05-04 22:06:25,901 - ALPHA_MIND - INFO - 2014-10-21 00:00:00 re-balance: 900 codes
2018-05-04 22:06:26,073 - ALPHA_MIND - INFO - 2014-11-04 00:00:00 re-balance: 900 codes
2018-05-04 22:06:26,252 - ALPHA_MIND - INFO - 2014-11-18 00:00:00 re-balance: 900 codes
2018-05-04 22:06:26,422 - ALPHA_MIND - INFO - 2014-12-02 00:00:00 re-balance: 900 codes
2018-05-04 22:06:26,593 - ALPHA_MIND - INFO - 2014-12-16 00:00:00 re-balance: 900 codes
2018-05-04 22:06:26,763 - ALPHA_MIND - INFO - 2014-12-30 00:00:00 re-balance: 900 codes
2018-05-04 22:06:26,935 - ALPHA_MIND - INFO - 2015-01-15 00:00:00 re-balance: 880 codes
2018-05-04 22:06:27,112 - ALPHA_MIND - INFO - 2015-01-29 00:00:00 re-balance: 879 codes
2018-05-04 22:06:27,286 - ALPHA_MIND - INFO - 2015-02-12 00:00:00 re-balance: 879 codes
2018-05-04 22:06:27,463 - ALPHA_

2018-05-04 22:07:09,293 - ALPHA_MIND - INFO - risk_model data loading finished ...
2018-05-04 22:07:37,234 - ALPHA_MIND - INFO - returns data loading finished ...
d:\ProgramData\Anaconda3\lib\site-packages\alpha_mind-0.1.1-py3.6-win-amd64.egg\alphamind\strategy\strategy.py:106: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  is_in_benchmark = (total_data.weight > 0.).astype(float).reshape((-1, 1))
2018-05-04 22:07:42,821 - ALPHA_MIND - INFO - 2010-01-04 00:00:00 re-balance: 798 codes
2018-05-04 22:07:42,872 - ALPHA_MIND - INFO - 2010-01-18 00:00:00 re-balance: 798 codes
d:\ProgramData\Anaconda3\lib\site-packages\alpha_mind-0.1.1-py3.6-win-amd64.egg\alphamind\strategy\strategy.py:142: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#

2018-05-04 22:07:56,286 - ALPHA_MIND - INFO - 2013-06-13 00:00:00 re-balance: 900 codes
2018-05-04 22:07:56,452 - ALPHA_MIND - INFO - 2013-06-27 00:00:00 re-balance: 900 codes
2018-05-04 22:07:56,620 - ALPHA_MIND - INFO - 2013-07-11 00:00:00 re-balance: 900 codes
2018-05-04 22:07:56,788 - ALPHA_MIND - INFO - 2013-07-25 00:00:00 re-balance: 900 codes
2018-05-04 22:07:56,960 - ALPHA_MIND - INFO - 2013-08-08 00:00:00 re-balance: 900 codes
2018-05-04 22:07:57,135 - ALPHA_MIND - INFO - 2013-08-22 00:00:00 re-balance: 900 codes
2018-05-04 22:07:57,300 - ALPHA_MIND - INFO - 2013-09-05 00:00:00 re-balance: 900 codes
2018-05-04 22:07:57,471 - ALPHA_MIND - INFO - 2013-09-23 00:00:00 re-balance: 899 codes
2018-05-04 22:07:57,645 - ALPHA_MIND - INFO - 2013-10-14 00:00:00 re-balance: 900 codes
2018-05-04 22:07:57,815 - ALPHA_MIND - INFO - 2013-10-28 00:00:00 re-balance: 900 codes
2018-05-04 22:07:57,987 - ALPHA_MIND - INFO - 2013-11-11 00:00:00 re-balance: 900 codes
2018-05-04 22:07:58,165 - ALPHA_

2018-05-04 22:08:12,393 - ALPHA_MIND - INFO - 2017-04-20 00:00:00 re-balance: 853 codes
2018-05-04 22:08:12,565 - ALPHA_MIND - INFO - 2017-05-05 00:00:00 re-balance: 854 codes
2018-05-04 22:08:12,743 - ALPHA_MIND - INFO - 2017-05-19 00:00:00 re-balance: 854 codes
2018-05-04 22:08:12,908 - ALPHA_MIND - INFO - 2017-06-06 00:00:00 re-balance: 854 codes
2018-05-04 22:08:13,074 - ALPHA_MIND - INFO - 2017-06-20 00:00:00 re-balance: 854 codes
2018-05-04 22:08:13,241 - ALPHA_MIND - INFO - 2017-07-04 00:00:00 re-balance: 855 codes
2018-05-04 22:08:13,418 - ALPHA_MIND - INFO - 2017-07-18 00:00:00 re-balance: 855 codes
2018-05-04 22:08:13,586 - ALPHA_MIND - INFO - 2017-08-01 00:00:00 re-balance: 853 codes
2018-05-04 22:08:13,757 - ALPHA_MIND - INFO - 2017-08-15 00:00:00 re-balance: 853 codes
2018-05-04 22:08:13,919 - ALPHA_MIND - INFO - 2017-08-29 00:00:00 re-balance: 853 codes
2018-05-04 22:08:14,084 - ALPHA_MIND - INFO - 2017-09-12 00:00:00 re-balance: 853 codes
2018-05-04 22:08:14,248 - ALPHA_

In [ ]:
target_vols = [0.015, 0.030, 0.045, 0.060]

with pd.ExcelWriter(f'{universe.name}_{benchmark_code}_tv.xlsx', engine='xlsxwriter') as writer:
    for i, target_vol in enumerate(target_vols):
        ret_df = create_scenario(0.01, target_vol=target_vol, method='tv')
        res_df = create_report(ret_df, 25)
        res_df.to_excel(writer, sheet_name=f'{i}')
        alpha_logger.info(f"target_vol: {target_vol:.4f} finished")